In [156]:
import time
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import datetime
import geopy.distance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


from scipy import stats

sns.set(color_codes=True)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load Extra TFT Data

In [160]:
tft_items = None
tft_traits = None
tft_champs = None
with open('./tft_extra_data/items.json') as json_file:
    tft_items = json.load(json_file)

with open('./tft_extra_data/traits.json') as json_file:
    tft_traits = json.load(json_file)

with open('./tft_extra_data/champions.json') as json_file:
    tft_champs = json.load(json_file)


# API Calls

In [128]:
URL = 'https://na1.api.riotgames.com/tft/league/v1/challenger'
api_key = 'RGAPI-a9c6b14d-2f84-4a3e-a7b8-df8f8b62da32'
headers = {'X-Riot-Token': api_key}


In [129]:
# Initial Challenger Data Pull
response = requests.get(URL, headers=headers)

In [14]:
summoner_names = []
league_points = []
for entry in response.json()['entries']:
    summoner_names.append(entry['summonerName'])
    league_points.append(entry['leaguePoints'])

In [34]:
player_df = pd.DataFrame(data = {'summoner_name': summoner_names, 'league_points': league_points})

In [35]:
player_df.sort_values(by=['league_points'], ascending=False, inplace=True)
player_df.reset_index(inplace=True)
player_df.drop(['index'], axis=1, inplace=True)

In [57]:
i = 0
player_uuids = []

In [70]:
URL_name = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/'

for index in range(i+1, len(player_df['summoner_name'])):
    name = player_df.iloc[index]['summoner_name']
    print(index, name)
    
    response = requests.get(URL_name + name, headers=headers)
    
    print(response.json()['puuid'])
    
    player_uuids.append(response.json()['puuid'])
    i = index



199 GreenbeingTFT
MmQ9S18WFQcAwNH1zAFuoiJeTKj9S0KYe6dl6Fe2YVuQkMvsNX3uHKYCM97gqlic-rHCiZE5czr4sQ


In [72]:
player_df['puuid'] = player_uuids

In [119]:
match_ids_df = pd.DataFrame(columns=['puuid', 'match_id'])
URL_match_ids = 'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/'
num_api_calls = 0
i = 0

In [120]:
for index in range(i, len(player_df['puuid'])):
    player_id = player_df.iloc[index]['puuid']
    if num_api_calls >= 100:
        time.sleep(120)
        num_api_calls = 0
    response = requests.get(URL_match_ids + player_id + '/ids?count=10', headers=headers)
    num_api_calls += 1

    for match in response.json():
        match_ids_df = match_ids_df.append({'puuid': player_id, 'match_id': match}, ignore_index=True)

    i = index
    

In [165]:
# Got match IDS
# Look up matches and obtain relevant information 
# Only those with top 4 placings?
game_info_df = pd.DataFrame(columns=['puuid', 
                                     'match_id', 
                                     'game_length', 
                                     'gold_left', 
                                     'last_round', 
                                     'level', 
                                     'placement',
                                     'players_eliminated',
                                     'time_eliminated',
                                     'total_damage_to_players',
                                     'traits', 
                                     'units'])
URL_game = 'https://americas.api.riotgames.com/tft/match/v1/matches/'
num_api_calls = 0
i = 0

In [ ]:
for match_id in match_ids_df['match_id']:
    if num_api_calls >= 100:
        time.sleep(120)
        num_api_calls = 0
    response = requests.get(URL_game + match_id, headers=headers)
    num_api_calls += 1
    data = response.json()['info']
    game_len = data['game_length']
    
    for p in data['participants']:
        traits = []
        units = []
        for t in p['traits']:
            traits.append((t['name'], t['tier_current']))
        
        for u in p['units']:
            units.append((u['name'], u['rarity'], u['tier'], u['items']))
        
        game_info_df = game_info_df.append({'puuid': p['puuid'],
                             'match_id': match_id,
                             'game_length': game_len,
                             'gold_left': p['gold_left'],
                             'last_round': p['last_round'],
                             'level': p['level'],
                             'placement': p['placement'],
                             'players_eliminated': p['players_eliminated'],
                             'time_eliminated': p['time_eliminated'],
                             'total_damage_to_players': p['total_damage_to_players'],
                             'traits': traits,
                             'units': units
                        }, ignore_index=True)
    print(len(game_info_df.index))

8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
240
248
256
264
272
280
288
296
304
312
320
328
336
344
352
360
368
376
384
392
400
408
416
424
432
440
448
456
464
472
480
488
496
504
512
520
528
536
544
552
560
568
576
584
592
600
608
616
624
632
640
648
656
664
672
680
688
696
704
712
720
728
736
744
752
760
768
776
784
792
800
808
816
824
832
840
848
856
864
872
880
888
896
904
912
920
928
936
944
952
960
968
976
984
992
1000
1008
1016
1024
1032
1040
1048
1056
1064
1072
1080
1088
1096
1104
1112
1120
1128
1136
1144
1152
1160
1168
1176
1184
1192
1200
1208
1216
1224
1232
1240
1248
1256
1264
1272
1280
1288
1296
1304
1312
1320
1328
1336
1344
1352
1360
1368
1376
1384
1392
1400
1408
1416
1424
1432
1440
1448
1456
1464
1472
1480
1488
1496
1504
1512
1520
1528
1536
1544
1552
1560
1568
1576
1584
1592
1600
1608
1616
1624
1632
1640
1648
1656
1664
1672
1680
1688
1696
1704
1712
1720
1728
1736
1744
1752
1760
1768
1776
1784
1792
1800
1808
1816
18

In [149]:
game_info_df.head(8)

,puuid,match_id,game_length,gold_left,last_round,level,placement,players_eliminated,time_eliminated,total_damage_to_players,traits,units
0,HbYsPpRS3X2RaQ8n5Sm9n2gKU80f927PzBk_krw4kdyRG6...,NA1_3249805878,1959.947266,5,31,8,4,0,1714.102783,95,"[(Berserker, 2)]","[(Renekton, 0, 2, [])]"
1,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249805878,1959.947266,0,35,9,2,3,1951.685059,146,"[(Avatar, 1)]","[(Annie, 3, 2, [])]"
2,HGeDGOQrklxGiScizUCLq_BVn5gTYBy-DldOuy38XcaKap...,NA1_3249805878,1959.947266,0,31,7,5,0,1711.923218,72,"[(Berserker, 0)]","[(Nocturne, 2, 2, [19, 19, 23])]"
3,VyCpwiJ3MifHEREwjj0cSAXHQIjFGd7UtWMBG9ZvCdlGso...,NA1_3249805878,1959.947266,30,26,7,7,0,1445.723633,55,"[(Inferno, 1)]","[(Kindred, 2, 2, [44, 22, 39])]"
4,UsiShYgxH3X5_pNeJjE-EpBhbR4vevorLcs83aI8ZSQFaZ...,NA1_3249805878,1959.947266,4,33,8,3,0,1819.271362,129,"[(Avatar, 1)]","[(Yasuo, 1, 2, [])]"
5,BXPswgAxjGfAfQHvaF-m7CtUdbBvZQhcviLqZwZDyDZbDn...,NA1_3249805878,1959.947266,50,28,8,6,0,1567.998779,80,"[(Inferno, 1)]","[(Nami, 4, 1, [25, 57])]"
6,zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMl...,NA1_3249805878,1959.947266,8,24,7,8,0,1324.369019,31,"[(Desert, 1)]","[(Aatrox, 2, 2, [])]"
7,BXZMHIfauv8M3BuaqO6wZnxQXrN2flWKj1GhJ6nNQeP3DD...,NA1_3249805878,1959.947266,1,35,9,1,4,1951.685059,176,"[(Avatar, 1)]","[(Malphite, 3, 2, [])]"
